這份 Notebook 示範 RAG from scratch (不使用向量資料庫)

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [2]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 44.9 MB/s eta 0:00:00


In [3]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

In [4]:
def get_embeddings(input, dimensions = 1536, model="text-embedding-3-small"):
  payload = { "input": input, "model": model, "dimensions": dimensions }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/embeddings', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["data"][0]["embedding"]
  else :
    return obj["error"]

## 上傳 PDF

104人力銀行《民生消費產業人才白皮書》.pdf
https://blog.104.com.tw/wp-content/uploads/2023/07/18152054/104%E4%BA%BA%E5%8A%9B%E9%8A%80%E8%A1%8C%E3%80%8A%E6%B0%91%E7%94%9F%E6%B6%88%E8%B2%BB%E7%94%A2%E6%A5%AD%E4%BA%BA%E6%89%8D%E7%99%BD%E7%9A%AE%E6%9B%B8%E3%80%8B-1.pdf

In [5]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

Saving 104人力銀行《民生消費產業人才白皮書》-1.pdf to 104人力銀行《民生消費產業人才白皮書》-1.pdf


In [6]:
import requests
import json
from pprint import pp
import fitz # PyMuPDF library

### 把 PDF 的純文字擷取出來

In [7]:
doc = fitz.open(file_name)
content = ''
for page in doc:
	text = page.get_text()
	content += text

In [9]:
content

'01序言\n序言\n104人力銀行  2023年7月\n缺工，已經影響了你我的生活！上館子，候位、點餐、等菜、茶水、結帳，只能等待。店家週轉率衝\n不高，不是沒客人，而是沒服務人員，只好門庭若市於外。怎麼解？\n104人力銀行連續第四年發表產業人才白皮書，不同於過去三年連續關注全球爆紅的半導體產業人\n才，今年，我們將觸角延伸到五大民生消費產業，包括餐飲業、零售業、批發業、住宿服務業、以及\n運動及旅遊休閒業，除了缺工已影響上千萬名消費者的日常，也因為五大民生消費產業的從業人員超\n過兩百二十萬人，全國平均每四位受僱者，就有一位來自民生消費產業，為數甚多。\n\xa0\n我們分析最近十年，共計8.6萬家民生消費產業的1,018萬筆徵才職缺，以及最近十年，六十三大產業\n共計777萬筆薪資資料，其中，近191萬筆薪資資料來自民生消費產業。撰寫過程中，一再出現的缺\n工，一再出現的低薪，已是眾所周知的艱困事實，找出背後的結構原因，提供人才職涯加值以及企業\n人才解方才是我們真正的數據責任(data responsibility)。\n\xa0\n令人安慰的是，民生消費產業雖然勞力密集，附加價值率低於科技製造業，但我們找出了八大薪資亮\n點職務、五大薪資強彈職務，雖僅是民生消費產業中的少數職務，但已驗證餐廚技術、專業研發、以\n及知識顧問仍可在低薪的民生消費產業中，因關鍵技術、因附加價值提高而讓薪資價格勝出。又如，\n我們已見到企業開始聘用中高齡填補人才，開始為了中高齡而進行職務再設計，同時調降兼職、調升\n正職的職缺比重，除了穩定人才來源，也創造職場安定，深度人才培力。然而，這些解方只能緩解局\n部的缺工壓力，如何有感加薪？如何留住並吸引人才？如何開挖現有人才？如何用對人提升附加價\n值？依舊是民生消費產業的經營關鍵和難題。\n\xa0\n為了讓數據更貼近真實，今年，我們仍持續訪談民生消費產業的人資招募主管、以及工作者，期望提\n供產官學研五大幫助：\n\xa0\n幫助企業攬才，協助企業多元徵才、開發人才，改善人才短缺。\n幫助企業留才，了解市場及區域薪資，搭配完整的薪資福利，留住人才。\n幫助職人判斷，透過企業人資長對人才的期望、以及職人的真實經歷，客觀呈現產業的工作樣貌。\n幫助職人發展，了解各地區徵才全貌、薪資水平、及入職應具備的技能及心理素質。\n幫助學校課程

### 來算算有多少 tokens

In [10]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.8 MB/s eta 0:00:00


In [11]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

In [12]:
num_tokens = len(tokenizer.encode(content))
print(num_tokens)

39997


## 自幹拆 chunks

這裡我們示範另一種拆 chunks 的寫法

In [13]:
# 參考自 https://platform.openai.com/docs/tutorials/web-qa-embeddings
import re
def split_into_many(text, max_tokens = 500):

    # 使用 re.split 來拆分字符串，並用 filter 函數去除空字符串
    pattern = r'[。.]' # 中文。和英文.
    sentences = list(filter(None, re.split(pattern, text)))

    # Get the number of tokens for each sentence
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]

    chunks = []
    tokens_so_far = 0
    chunk = []

    # Loop through the sentences and tokens joined together in a tuple
    for sentence, token in zip(sentences, n_tokens):

        # If the number of tokens so far plus the number of tokens in the current sentence is greater
        # than the max number of tokens, then add the chunk to the list of chunks and reset
        # the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0

        # If the number of tokens in the current sentence is greater than the max number of
        # tokens, go to the next sentence
        if token > max_tokens:
            continue

        # Otherwise, add the sentence to the chunk and add the number of tokens to the total
        chunk.append(sentence)
        tokens_so_far += token + 1

    return chunks


In [14]:
docs = split_into_many(content)

In [15]:
len(docs)

90

## 自幹計算相似度

這裡我們在示範另一種寫法，使用 sklearn 提供的 cosine_similarity 方法

In [16]:
embeddings = [] # 這個存所有當作索引的向量
for doc in docs:
  embeddings.append( get_embeddings(doc) )

In [17]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 參數 list_of_doc_vectors 是所有文件的 embeddings 向量
# 參數 query_vector 是查詢字串的 embedding 向量
# 參數 top_k 是回傳的比數
def get_top_k_indices(list_of_doc_vectors, query_vector, top_k):
  # 轉成 numpy arrays
  list_of_doc_vectors = np.array(list_of_doc_vectors)
  query_vector = np.array(query_vector)

  # 計算 cosine similarities
  similarities = cosine_similarity(query_vector.reshape(1, -1), list_of_doc_vectors).flatten()

  # 根據 cosine similarity 排序
  sorted_indices = np.argsort(similarities)[::-1]

  # 取出 top K 的索引數字
  top_k_indices = sorted_indices[:top_k]

  return top_k_indices

In [18]:
def search_similar(embeddings, query, n=3):
    query_embedding = get_embeddings(query)

    best_indexes = get_top_k_indices(embeddings, query_embedding, n)
    best_k_chunks = [docs[i] for i in best_indexes]

    return best_k_chunks

## 開始 QA

先搜尋最相似的 chunk 做為 context 參考資料

In [19]:
question = "今年的徵才趨勢什麼?"
chunks = search_similar(embeddings, question)

In [24]:
context = ''
for chunk in chunks:
  context += f"{chunk} \n----------------\n"

In [25]:
print(context)

7%
vs 2022 Q1 +12. 1%
年增幅
徵才人數
兼職佔比
全職佔比
徵才人數
兼職佔比
全職佔比
運動及旅遊休閒業近十年徵才趨勢
資料來源：104人力銀行 / 資料說明：運動及旅遊休閒業於2013~2023年每季月平均徵才人數及招募全職、兼職占比
0
5,000
10,000
15,000
20,000
25,000
30,000
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
12,152
16,326
19,417
19,417
26,460
26,460
75%
75%
25%
25%
vs 2020 Q1 +117. 8%
vs 2022 Q1 +36. 
----------------
4%
年增幅
42%
42%
58%
58%
138,160
138,160
11
徵才趨勢
資料來源：104人力銀行 / 資料說明：批發業於2013~2023年每季月平均徵才人數及招募全職、兼職占比
批發業近十年徵才趨勢
住宿服務業近十年徵才趨勢
資料來源：104人力銀行 / 資料說明：住宿服務業於2013~2023年每季月平均徵才人數及招募全職、兼職占比
0
5,000
10,000
15,000
20,000
25,000
30,000
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
80%
80%
12,506
15,997
20,135
20,135
20%
20%
26,653
26,653
vs 2020 Q1 +113. 3%
vs 2022 Q1 +32. 
----------------
1萬人，當時創下2014年
第三季以來的七年新低. 隨著疫情趨緩、市場解封、民間消費持續擴張，自2021年第四季起，徵
才規模已超越疫情之前的水準. 截至2023年第一季平均每月徵才38. 2萬人，比2020年第二季谷
底17. 1萬人大增122. 6%，三年同期增幅90. 9%、單年同期增幅23. 3%. 
民生消費產業的人才缺口持續擴大，2023年第一季「求供比」高達8. 0，平均每位想進入民生消
費產業的求職者可分到8個民生消費產業的工作機會，遠高於整體市場的1. 8，「事浮於人」民生
消費產業的企業徵才難度是整體企業的4. 
----------------



### 組出 prompt

In [27]:
prompt= f'''
Answer the question based on the context below,
and if the question can't be answered based on the context, say "I don't know"

Context:

{context}

Question: ```{question}```
Answer:'''

print(prompt)


Answer the question based on the context below,
and if the question can't be answered based on the context, say "I don't know"

Context: 

7%
vs 2022 Q1 +12. 1%
年增幅
徵才人數
兼職佔比
全職佔比
徵才人數
兼職佔比
全職佔比
運動及旅遊休閒業近十年徵才趨勢
資料來源：104人力銀行 / 資料說明：運動及旅遊休閒業於2013~2023年每季月平均徵才人數及招募全職、兼職占比
0
5,000
10,000
15,000
20,000
25,000
30,000
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
12,152
16,326
19,417
19,417
26,460
26,460
75%
75%
25%
25%
vs 2020 Q1 +117. 8%
vs 2022 Q1 +36. 
----------------
4%
年增幅
42%
42%
58%
58%
138,160
138,160
11
徵才趨勢
資料來源：104人力銀行 / 資料說明：批發業於2013~2023年每季月平均徵才人數及招募全職、兼職占比
批發業近十年徵才趨勢
住宿服務業近十年徵才趨勢
資料來源：104人力銀行 / 資料說明：住宿服務業於2013~2023年每季月平均徵才人數及招募全職、兼職占比
0
5,000
10,000
15,000
20,000
25,000
30,000
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%
80%
80%
12,506
15,997
20,135
20,135
20%
20%
26,653
26,653
vs 2020 Q1 +113. 3%
vs 2022 Q1 +32. 
----------------
1萬人，當時創下2014年
第三季以來的七年新低. 隨著疫情趨緩、市場解封、民間消費持續擴張，自2021年第四季起，徵
才規模已超越疫情之前的水準. 截至2023年第一季平均每月徵才38. 2萬人，比2020年第二季谷
底17. 1萬人大增122. 6%，三年同期增幅90. 9%、單年同期增幅23. 3%. 
民生消費

### 問 LLM

In [28]:
result = get_completion([ {"role": "user", "content": prompt }], model="gpt-4-turbo-preview")
print(result)

今年的徵才趨勢顯示民生消費產業的人才缺口持續擴大，2023年第一季「求供比」高達8.0，意味著平均每位想進入民生消費產業的求職者可分到8個工作機會，遠高於整體市場的1.8。这表明民生消費產業的企業徵才難度是整體企業的4倍，顯示出一個強烈的徵才需求增長。
